# Feature Engineering Pipeline - FIXED (No Data Leakage)

**Problem:** Oryginalny notebook obliczał cechy używając WSZYSTKICH transakcji, włącznie z transakcją którą próbujemy przewidzieć.

**Rozwiązanie:**
- **User features:** Obliczamy z POPRZEDNICH transakcji tego użytkownika (cumulative)
- **Product/Category features:** Obliczamy globalnie, ale wykluczamy aktualną transakcję
- **RFM features:** Tylko frequency (bez monetary, który zawiera Purchase)

**Podejście:**
1. Dodajemy `transaction_id` jako kolejność transakcji dla każdego użytkownika
2. Używamy Window Functions z `rowsBetween(unboundedPreceding, -1)` dla user features
3. Dla product/category: obliczamy total stats, potem odejmujemy aktualną transakcję

---

In [1]:
# Imports
import sys
sys.path.append('../')

from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, count, sum as _sum, avg, min as _min, max as _max,
    countDistinct, stddev, when, lit, dense_rank, percent_rank,
    broadcast, row_number, lag, monotonically_increasing_id
)
from pyspark.sql.window import Window
from pyspark.ml.feature import (
    StringIndexer, OneHotEncoder, VectorAssembler,
    StandardScaler
)
from pyspark.ml import Pipeline

from config.spark_config import SparkConfig

import warnings
warnings.filterwarnings('ignore')

## 1. Start Spark Session

In [2]:
spark = SparkConfig.get_spark_session("BlackFriday-FeatureEngineering-FIXED")
print("\nSpark session ready!")

✓ Windows: HADOOP_HOME set to C:\Users\usate\PycharmProjects\BlackFriday\hadoop
  Note: If you encounter permission errors, download winutils.exe
  from https://github.com/steveloughran/winutils and place in hadoop/bin/
Spark Session Created: BlackFriday-FeatureEngineering-FIXED
Spark Version: 3.5.3
Master: local[*]
App Name: BlackFriday-FeatureEngineering-FIXED

Spark session ready!


## 2. Load Raw Data

In [3]:
# Load raw data
df_raw = spark.read.csv(
    "../data/raw/train.csv",
    header=True,
    inferSchema=True
)

print(f"Dataset shape: {df_raw.count():,} rows x {len(df_raw.columns)} columns")
df_raw.printSchema()

Dataset shape: 550,068 rows x 12 columns
root
 |-- User_ID: integer (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Occupation: integer (nullable = true)
 |-- City_Category: string (nullable = true)
 |-- Stay_In_Current_City_Years: string (nullable = true)
 |-- Marital_Status: integer (nullable = true)
 |-- Product_Category_1: integer (nullable = true)
 |-- Product_Category_2: integer (nullable = true)
 |-- Product_Category_3: integer (nullable = true)
 |-- Purchase: integer (nullable = true)



## 3. Data Cleansing

In [4]:
# Clean data
df_clean = df_raw.dropDuplicates()

# Fill missing Product_Category_2 and Product_Category_3 with 0
df_clean = df_clean.fillna({
    'Product_Category_2': 0,
    'Product_Category_3': 0
})

print(f"Cleaned data: {df_clean.count():,} rows")

Cleaned data: 550,068 rows


## 4. Add Transaction Ordering

**Problem:** Dataset nie ma timestampów.

**Rozwiązanie:** Dodajemy `transaction_order` dla każdego użytkownika używając `row_number()`.
- Sortujemy po User_ID i używamy row_number() jako proxy dla czasu
- Dla każdej transakcji, będziemy obliczać cechy z POPRZEDNICH transakcji (lower transaction_order)

In [5]:
# Add transaction order for each user
# We'll use row_number() partitioned by User_ID as a proxy for transaction time
window_user_order = Window.partitionBy("User_ID").orderBy(monotonically_increasing_id())

df_clean = df_clean.withColumn(
    "transaction_order",
    row_number().over(window_user_order)
)

print("Transaction order added")
df_clean.select("User_ID", "Product_ID", "Purchase", "transaction_order").show(10)

Transaction order added
+-------+----------+--------+-----------------+
|User_ID|Product_ID|Purchase|transaction_order|
+-------+----------+--------+-----------------+
|1000004| P00025442|   19693|                1|
|1000004| P00217442|   15499|                2|
|1000004|  P0097242|   15686|                3|
|1000004| P00184942|   19215|                4|
|1000004| P00329542|    3849|                5|
|1000004| P00114942|   19120|                6|
|1000004| P00318742|   15853|                7|
|1000004| P00375436|     481|                8|
|1000004| P00112142|   11765|                9|
|1000004| P00346142|   15854|               10|
+-------+----------+--------+-----------------+
only showing top 10 rows



## 5. User Features (Cumulative - No Leakage)

**Key Change:** Używamy `rowsBetween(Window.unboundedPreceding, -1)` aby wykluczyć aktualną transakcję.

Dla każdej transakcji użytkownika, obliczamy:
- user_purchase_count: liczba POPRZEDNICH transakcji
- user_avg_purchase: średnia z POPRZEDNICH transakcji
- user_total_spent: suma POPRZEDNICH transakcji
- etc.

**Uwaga:** Pierwsza transakcja użytkownika będzie miała NULL dla tych cech (bo nie ma poprzednich).

In [6]:
# Define window for cumulative user features
# Key: rowsBetween(unboundedPreceding, -1) excludes current row!
window_user = Window.partitionBy("User_ID").orderBy("transaction_order").rowsBetween(Window.unboundedPreceding, -1)

# User aggregations (cumulative, excluding current transaction)
df_enriched = df_clean.withColumn(
    "user_purchase_count_prior",
    count("Purchase").over(window_user)
).withColumn(
    "user_total_spent_prior",
    _sum("Purchase").over(window_user)
).withColumn(
    "user_avg_purchase_prior",
    avg("Purchase").over(window_user)
).withColumn(
    "user_min_purchase_prior",
    _min("Purchase").over(window_user)
).withColumn(
    "user_max_purchase_prior",
    _max("Purchase").over(window_user)
).withColumn(
    "user_std_purchase_prior",
    stddev("Purchase").over(window_user)
)

# Count distinct products (prior)
# Note: countDistinct doesn't work with window rowsBetween, so we'll calculate differently
# For now, we'll skip unique products/categories or use total count as proxy

print("✓ User cumulative features added (excluding current transaction)")
print("\nSample (first few transactions for each user):")
df_enriched.select(
    "User_ID", "transaction_order", "Purchase",
    "user_purchase_count_prior", "user_avg_purchase_prior", "user_total_spent_prior"
).orderBy("User_ID", "transaction_order").show(15)

✓ User cumulative features added (excluding current transaction)

Sample (first few transactions for each user):
+-------+-----------------+--------+-------------------------+-----------------------+----------------------+
|User_ID|transaction_order|Purchase|user_purchase_count_prior|user_avg_purchase_prior|user_total_spent_prior|
+-------+-----------------+--------+-------------------------+-----------------------+----------------------+
|1000001|                1|   10900|                        0|                   NULL|                  NULL|
|1000001|                2|    8839|                        1|                10900.0|                 10900|
|1000001|                3|    9946|                        2|                 9869.5|                 19739|
|1000001|                4|    2763|                        3|                 9895.0|                 29685|
|1000001|                5|   15416|                        4|                 8112.0|                 32448|
|100000

In [7]:
# Fill nulls for first transaction of each user
# First transaction has no prior history, so we fill with defaults
df_enriched = df_enriched.fillna({
    'user_purchase_count_prior': 0,
    'user_total_spent_prior': 0,
    'user_avg_purchase_prior': 0,
    'user_min_purchase_prior': 0,
    'user_max_purchase_prior': 0,
    'user_std_purchase_prior': 0
})

print("✓ Filled nulls for first transactions")

✓ Filled nulls for first transactions


## 6. Product Features (Excluding Current Transaction)

**Approach:** 
1. Calculate total product stats (all transactions)
2. For each row, subtract current transaction to get "excluding current"

**Formula:**
```
product_avg_price_excl = (product_total_revenue - current_Purchase) / (product_count - 1)
```

In [8]:
# Calculate TOTAL product stats (including all transactions)
product_features_total = df_clean.groupBy("Product_ID").agg(
    count("*").alias("product_total_count"),
    _sum("Purchase").alias("product_total_revenue_all"),
    countDistinct("User_ID").alias("product_unique_users")
)

# Join back to main dataset
df_enriched = df_enriched.join(
    broadcast(product_features_total),
    on="Product_ID",
    how="left"
)

# Calculate product stats EXCLUDING current transaction
df_enriched = df_enriched.withColumn(
    "product_purchase_count",
    col("product_total_count") - 1  # Exclude current
).withColumn(
    "product_total_revenue",
    col("product_total_revenue_all") - col("Purchase")  # Exclude current purchase
).withColumn(
    "product_avg_price",
    when(col("product_total_count") > 1,
         (col("product_total_revenue_all") - col("Purchase")) / (col("product_total_count") - 1)
    ).otherwise(0)  # If only 1 purchase, no prior data
)

# Product popularity (based on count, not price)
window_product_pop = Window.orderBy(col("product_purchase_count").desc())
df_enriched = df_enriched.withColumn(
    "product_popularity_score",
    percent_rank().over(window_product_pop)
)

print("✓ Product features added (excluding current transaction)")

✓ Product features added (excluding current transaction)


## 7. Category Features (Excluding Current Transaction)

In [9]:
# Calculate TOTAL category stats
category_features_total = df_clean.groupBy("Product_Category_1").agg(
    count("*").alias("category_total_count"),
    _sum("Purchase").alias("category_total_revenue_all")
)

# Join back
df_enriched = df_enriched.join(
    broadcast(category_features_total),
    on="Product_Category_1",
    how="left"
)

# Calculate category stats EXCLUDING current
df_enriched = df_enriched.withColumn(
    "category_purchase_count",
    col("category_total_count") - 1
).withColumn(
    "category_avg_price",
    when(col("category_total_count") > 1,
         (col("category_total_revenue_all") - col("Purchase")) / (col("category_total_count") - 1)
    ).otherwise(0)
)

print("✓ Category features added (excluding current transaction)")

✓ Category features added (excluding current transaction)


## 8. RFM Features (Frequency Only - No Monetary!)

**Change:** We ONLY use frequency (purchase count), NOT monetary (total spent).

Why? Monetary uses Purchase values, which causes data leakage.

In [10]:
# RFM: Only Frequency (based on prior purchase count)
df_enriched = df_enriched.withColumn(
    "rfm_frequency_score",
    percent_rank().over(Window.orderBy(col("user_purchase_count_prior")))
)

# NOTE: We do NOT calculate rfm_monetary_score or rfm_score
# because they would use Purchase values (data leakage)

print("✓ RFM Frequency score added (NO monetary score - avoiding leakage)")

✓ RFM Frequency score added (NO monetary score - avoiding leakage)


## 9. Interaction Features (Safe Only)

**Changes:**
- ❌ REMOVED: `purchase_vs_user_avg_ratio` (uses Purchase / user_avg)
- ❌ REMOVED: `purchase_vs_product_avg_ratio` (uses Purchase / product_avg)
- ❌ REMOVED: `is_above_user_avg` (compares Purchase with avg)
- ✅ KEPT: `is_high_value_customer` (based on frequency only)

In [11]:
# High value customer flag (based on frequency, not monetary)
df_enriched = df_enriched.withColumn(
    "is_high_value_customer",
    when(col("rfm_frequency_score") >= 0.8, 1).otherwise(0)
)

# Derived features (safe)
df_enriched = df_enriched.withColumn(
    "user_purchase_range_prior",
    col("user_max_purchase_prior") - col("user_min_purchase_prior")
)

print("✓ Safe interaction features added")

✓ Safe interaction features added


## 10. Categorical Encoding

In [12]:
# Define categorical columns
categorical_cols = [
    "Gender",
    "Age",
    "City_Category",
    "Stay_In_Current_City_Years"
]

# String Indexing
indexers = [
    StringIndexer(inputCol=col, outputCol=col + "_index", handleInvalid="keep")
    for col in categorical_cols
]

# One-Hot Encoding
encoders = [
    OneHotEncoder(inputCol=col + "_index", outputCol=col + "_vec", dropLast=False)
    for col in categorical_cols
]

# Create pipeline
encoding_pipeline = Pipeline(stages=indexers + encoders)

# Fit and transform
encoding_model = encoding_pipeline.fit(df_enriched)
df_encoded = encoding_model.transform(df_enriched)

print("✓ Categorical encoding completed")

✓ Categorical encoding completed


## 11. Feature Selection for ML

**Key changes:**
- Use `_prior` versions of user features (computed from previous transactions only)
- Use product/category features that exclude current transaction
- NO monetary-based RFM features

In [13]:
# Select features for ML models
feature_cols = [
    # Categorical (indexed)
    "Gender_index", "Age_index", "City_Category_index",
    "Stay_In_Current_City_Years_index",
    
    # Numerical demographics
    "Occupation", "Marital_Status",
    
    # Product categories
    "Product_Category_1", "Product_Category_2", "Product_Category_3",
    
    # User features (PRIOR - from previous transactions)
    "user_purchase_count_prior",
    "user_total_spent_prior",
    "user_avg_purchase_prior",
    "user_std_purchase_prior",
    "user_min_purchase_prior",
    "user_max_purchase_prior",
    "user_purchase_range_prior",
    
    # Product features (excluding current)
    "product_purchase_count",
    "product_unique_users",
    "product_avg_price",
    "product_popularity_score",
    
    # Category features (excluding current)
    "category_purchase_count",
    "category_avg_price",
    
    # RFM features (frequency only)
    "rfm_frequency_score",
    
    # Interaction features
    "is_high_value_customer"
]

print(f"Total features: {len(feature_cols)}")
print("\nFeatures:")
for i, f in enumerate(feature_cols, 1):
    print(f"  {i}. {f}")

Total features: 24

Features:
  1. Gender_index
  2. Age_index
  3. City_Category_index
  4. Stay_In_Current_City_Years_index
  5. Occupation
  6. Marital_Status
  7. Product_Category_1
  8. Product_Category_2
  9. Product_Category_3
  10. user_purchase_count_prior
  11. user_total_spent_prior
  12. user_avg_purchase_prior
  13. user_std_purchase_prior
  14. user_min_purchase_prior
  15. user_max_purchase_prior
  16. user_purchase_range_prior
  17. product_purchase_count
  18. product_unique_users
  19. product_avg_price
  20. product_popularity_score
  21. category_purchase_count
  22. category_avg_price
  23. rfm_frequency_score
  24. is_high_value_customer


In [14]:
# Fill any remaining nulls with 0
df_encoded = df_encoded.fillna(0, subset=feature_cols)

# Vector Assembler
assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features",
    handleInvalid="skip"
)

df_features = assembler.transform(df_encoded)

print(f"✓ Feature vector assembled with {len(feature_cols)} features")

✓ Feature vector assembled with 24 features


## 12. Feature Scaling

In [15]:
# Standard Scaler
scaler = StandardScaler(
    inputCol="features",
    outputCol="scaled_features",
    withMean=False,
    withStd=True
)

scaler_model = scaler.fit(df_features)
df_scaled = scaler_model.transform(df_features)

print("✓ Features scaled")

✓ Features scaled


## 13. Final Feature Set

In [16]:
# Select final columns
final_cols = [
    # IDs
    "User_ID", "Product_ID",
    
    # Target
    "Purchase",
    
    # Original features
    "Gender", "Age", "Occupation", "City_Category",
    "Stay_In_Current_City_Years", "Marital_Status",
    "Product_Category_1", "Product_Category_2", "Product_Category_3",
    
    # Transaction order
    "transaction_order",
    
    # Engineered features (prior/excluding current)
    "user_purchase_count_prior", "user_total_spent_prior", "user_avg_purchase_prior",
    "user_std_purchase_prior", "user_min_purchase_prior", "user_max_purchase_prior",
    "user_purchase_range_prior",
    "product_purchase_count", "product_unique_users", "product_avg_price",
    "product_popularity_score",
    "category_purchase_count", "category_avg_price",
    "rfm_frequency_score", "is_high_value_customer",
    
    # ML-ready features
    "features", "scaled_features"
]

df_final = df_scaled.select(*final_cols)

print("="*60)
print("FEATURE ENGINEERING COMPLETE (NO DATA LEAKAGE)")
print("="*60)
print(f"Final dataset: {df_final.count():,} rows x {len(df_final.columns)} columns")
print(f"Total features in vector: {len(feature_cols)}")
print("\n✅ All features computed WITHOUT using current transaction!")
print("="*60)

FEATURE ENGINEERING COMPLETE (NO DATA LEAKAGE)
Final dataset: 550,068 rows x 30 columns
Total features in vector: 24

✅ All features computed WITHOUT using current transaction!


In [17]:
# Show sample
print("\nSample data (first user's transactions):")
df_final.filter(col("User_ID") == 1000001).select(
    "User_ID", "transaction_order", "Purchase",
    "user_purchase_count_prior", "user_avg_purchase_prior",
    "product_avg_price", "category_avg_price"
).show(10, truncate=False)


Sample data (first user's transactions):
+-------+-----------------+--------+-------------------------+-----------------------+------------------+------------------+
|User_ID|transaction_order|Purchase|user_purchase_count_prior|user_avg_purchase_prior|product_avg_price |category_avg_price|
+-------+-----------------+--------+-------------------------+-----------------------+------------------+------------------+
|1000001|1                |10900   |0                        |0.0                    |11445.199057714959|10096.665990500693|
|1000001|2                |8839    |1                        |10900.0                |6975.126192223038 |6240.070959107413 |
|1000001|3                |9946    |2                        |9869.5                 |8014.921458625526 |7498.9365980829325|
|1000001|4                |2763    |3                        |9895.0                 |2790.9124497991966|2329.622617426821 |
|1000001|5                |15416   |4                        |8112.0               

## 14. Validation: Check for Data Leakage

Let's verify that our features don't leak information about the target.

In [18]:
print("="*60)
print("VALIDATION: Checking for Data Leakage")
print("="*60)

# Check 1: For first transaction of each user, prior features should be 0
first_transactions = df_final.filter(col("transaction_order") == 1)
print(f"\n✓ Check 1: First transactions for each user")
print(f"  Count: {first_transactions.count():,}")
print("  Prior features should be 0:")
first_transactions.select(
    "User_ID", "Purchase",
    "user_purchase_count_prior", "user_avg_purchase_prior"
).show(5)

# Check 2: For subsequent transactions, prior avg should NOT equal current Purchase
subsequent_transactions = df_final.filter(col("transaction_order") > 1).sample(False, 0.001)
print("\n✓ Check 2: Subsequent transactions (sample)")
print("  user_avg_purchase_prior should NOT equal Purchase:")
subsequent_transactions.select(
    "User_ID", "transaction_order", "Purchase",
    "user_avg_purchase_prior"
).withColumn(
    "are_equal",
    when(col("Purchase") == col("user_avg_purchase_prior"), "EQUAL").otherwise("DIFFERENT")
).show(10)

print("\n✅ Validation complete!")
print("="*60)

VALIDATION: Checking for Data Leakage

✓ Check 1: First transactions for each user
  Count: 5,891
  Prior features should be 0:
+-------+--------+-------------------------+-----------------------+
|User_ID|Purchase|user_purchase_count_prior|user_avg_purchase_prior|
+-------+--------+-------------------------+-----------------------+
|1003034|    8905|                        0|                    0.0|
|1003020|    8732|                        0|                    0.0|
|1006015|    8755|                        0|                    0.0|
|1004046|    8888|                        0|                    0.0|
|1003229|    8895|                        0|                    0.0|
+-------+--------+-------------------------+-----------------------+
only showing top 5 rows


✓ Check 2: Subsequent transactions (sample)
  user_avg_purchase_prior should NOT equal Purchase:
+-------+-----------------+--------+-----------------------+---------+
|User_ID|transaction_order|Purchase|user_avg_purchase_pri

## 15. Save to Delta Lake

In [19]:
# Save to Delta Lake (separate location to avoid overwriting original)
delta_path = "../data/processed/delta/features_fixed"

df_final.write \
    .format("delta") \
    .mode("overwrite") \
    .save(delta_path)

print(f"\n✓ Data saved to Delta Lake: {delta_path}")
print("\n" + "="*60)
print("✅ FEATURE ENGINEERING FIXED - NO DATA LEAKAGE")
print("="*60)
print("\n📊 Summary of Changes:")
print("  ✓ User features: computed from PRIOR transactions only")
print("  ✓ Product features: exclude current transaction")
print("  ✓ Category features: exclude current transaction")
print("  ✓ RFM: frequency only (NO monetary)")
print("  ✓ Removed: purchase_vs_*_ratio features (contained Purchase)")
print("\n🎯 Next Step: Train models on features_fixed dataset")
print("="*60)


✓ Data saved to Delta Lake: ../data/processed/delta/features_fixed

✅ FEATURE ENGINEERING FIXED - NO DATA LEAKAGE

📊 Summary of Changes:
  ✓ User features: computed from PRIOR transactions only
  ✓ Product features: exclude current transaction
  ✓ Category features: exclude current transaction
  ✓ RFM: frequency only (NO monetary)
  ✓ Removed: purchase_vs_*_ratio features (contained Purchase)

🎯 Next Step: Train models on features_fixed dataset
